# Transformers

In this notebook, we are going to define and implement a Transformer model.

In [29]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

## Scaled Dot-Product Attention

<img src='assets/1/scaled-dot-product-3.PNG'/> 

In [41]:
# Scaled Dot-Product Attention calculation 
# from the paper (section 3.2.1 Scaled Dot-Product Attention):
# 'We compute the dot products of the query with all keys, divide each by √dk, 
# and apply a softmax function to obtain the weights on the values.'

class ScaledDotProductAttention(nn.Module):
    def __init__(self):
        super(ScaledDotProductAttention, self).__init__()
        
    def forward(self, Q, K, V, mask=False):
        
        # compute dot product of the all query with all keys
        dot_products = torch.matmul(Q, torch.transpose(K, 0, 1)) 
        
        # divide each by √dk
        d_k = K.shape[1]    # get length of key vector
        scaled_dot_products = dot_products / np.sqrt(d_k)
        
        # apply a softmax function to obtain weights on values
        weights = F.softmax(scaled_dot_products, dim=-1)
        
        # get weighted values by multiplying values with weights(softmax scores)
        weighted_values = torch.matmul(weights, V)
        
        # apply mask to prevent positions from attending to subsequent positions in decoder
        if mask==True:
            pass
            # look_ahead_mask = np.where(mask_ind, 1, -np.inf)
            # weighted_values = weighted_values * look_ahead_mask  
        
        return weighted_values
    

Scaled Dot-product attention is identical to the [Dot-product attention algorithm](https://arxiv.org/pdf/1508.04025.pdf), except for the scaling  factor of 1/√dk.
>For large values of dk, the dot products grow large in magnitude, pushing the softmax function into regions where it has extremely small gradients. To counteract this effect, we scale the dot products by 1/√dk. <sub>-from section 3.2.1 Scaled Dot-Product Attention<sub>

**Masked Multi-Head Attention**

<img src='assets/1/masked-attention.PNG' width=50% height=50% /> 

>We need to prevent leftward information flow in the decoder to preserve the auto-regressive property. We **implement this inside of scaled dot-product attention** by masking out (setting to −∞) all values in the input of the softmax which correspond to illegal connections. <sub>-from section 3.2.3 Applications of Attention in our Model<sub>

>We also modify the self-attention sub-layer in the decoder stack to prevent positions from attending to subsequent positions. This masking, combined with fact that the output embeddings are offset by one position, ensures that the predictions for position i can depend only on the known outputs at positions less than i. <sub>-from section 3.1 Encoder and Decoder Stacks<sub>
    
Auto-regressive property can be defined as to predict future values based on past value. Therefore we need to mask the inputs that are subsequent to that position. So that, with having data given in parallel, the decoder will not learn a simple mapping provided all target outputs.

Let's do a unit test on Scaled Dot-Product Attention implementation.

<img src='assets/1/scaled-dot-product-matrix.PNG' /> 

In [42]:
# define mock data assuming n=3 embeddings
n = 3
d_k = 64

# define K,Q and V matrices
K = torch.randn(n*d_k).reshape(n, d_k)
Q = torch.randn(n*d_k).reshape(n, d_k)
V = torch.randn(n*d_k).reshape(n, d_k)

scaled_dot_product = ScaledDotProductAttention()
output = scaled_dot_product(Q, K, V)    # apply scaled-dot product attention

output.shape

torch.Size([3, 64])

In [ ]:
# masked_output = scaled_dot_product(Q, K, V, mask=True)
# masked_output

## Attention

<img src='assets/1/attention-3.PNG'/> 

In [43]:
class Attention(nn.Module):
    def __init__(self, d_model=512, d_k=64):
        super(Attention, self).__init__()
        
        # define Key, Query and Value weight matrices
        self.WK = nn.Parameter(torch.randn(d_model, d_k))
        self.WQ = nn.Parameter(torch.randn(d_model, d_k))
        self.WV = nn.Parameter(torch.randn(d_model, d_k))
        
        self.scaled_dot_product = ScaledDotProductAttention()
        
    def forward(self, inputs, mask=False):
        WK, WQ, WV = self.WK, self.WQ, self.WV
        
        # apply linear transformations
        
        # get packed Key, Query and Value matrices
        # by multiplying inputs with K, Q and V   
        K = torch.matmul(inputs, WK)
        Q = torch.matmul(inputs, WQ)
        V = torch.matmul(inputs, WV)
        
        
        # apply scaled dot-product attention
        attention_scores = self.scaled_dot_product(Q, K, V, mask)
        
        return attention_scores

>An attention function can be described as mapping a query and a set of key-value pairs to an output, where the query, keys, values, and output are all vectors. The output is computed as a weighted sum of the values, where the weight assigned to each value is computed by a compatibility function of the query with the corresponding key. <sub>-from section 3.2 Attention<sub>

>In practice, we compute the attention function on a set of queries simultaneously, packed together
into a matrix Q. The keys and values are also packed together into matrices K and V. <sub>-from section 3.2.1 Scaled Dot-Product Attention<sub>
    
To find a mapping between an input and output with respect to key-value pairs and a query, we define learnable parameters: key, query and value **weight matrices**. Linear transformation on input happens by multiplying input with Q, K and V weight matrices.

Let's do a unit test on Attention implementation.

<img src='assets/1/attention-matrix.PNG'/>

In [44]:
n = 3   # input embeddings
d_model = 512  # embedding dimensions
d_k = 64  # key dimension

# define input
X = torch.randn(n*d_model).reshape(n, d_model)

# init attention 
attention = Attention(d_model, d_k)

# compute attention scores
attention_scores = attention(X)

attention_scores.shape

torch.Size([3, 64])

## Multi-Head Attention

<img src='assets/1/multi-head-attention-3.PNG' width=75% height=75%/>

In [ ]:
# Multi-Head Attention consists of several attention layers running in parallel.       
class MultiHeadAttention(nn.Module):
    
    def __init__(self, h=8, d_model=512, d_k=64):
        # add multi-head attention layers to a sequential container
        multi_head_attention = []
        
        # define linear layer 
        self.W = nn.Parameter(torch.randn(h*d_k, d_model))
    
    def forward(self):
        
        # apply attention
        
        # collect attentions
        
        # concat attentions
        attentions = torch.cat((x, x, x), 1)
        
        # apply linear transformation
        multi_attentions = np.matmul(z, W)
        # reduce dimensionality

Let's do a unit test on Multi-Head Attention implementation.

<img src='assets/1/multi-head-matrix.PNG'/>

## Feed Forward Neural Network
<img src='assets/1/ffnn.PNG' width=75% height=75%/>


In [ ]:
 # feedforward neural network here
class FeedForwardNetwork(nn.Module):
    def __init__(self, d_ff=2048):
        super(FeedForwardNetwork, self).__init__()
        self.fc1 = nn.Linear()
        self.fc2 = nn.Linear()

    def forward(self, x):
        
  

In [ ]:
# Let's not skip the unit test of FeedForward network. 

## Sublayer

In [ ]:
# We employ a residual connection around each of the two sub-layers, 
# followed by layer normalization. That is'The output of each sub-layer is
# LayerNorm(x + Sublayer(x)), where Sublayer(x) is
# the function implemented by the sub-layer itself.'
# from section - 3.1 Encoder and Decoder Stacks

# define residual learning block
class Residual_Connection(nn.Module):
    def __init__(self, layer):
        super(Residual_Connection, self).__init__()
        self.layer = layer     # function implemented by the sublayer

    def forward(self, x):
        f_x = self.layer(x)    # apply Sublayer(x)
        x = x + f_x            # x + Sublayer(x)
        return x
    
class Sublayer(nn.Module):
    def __init__(self, layer):
        super(SubLayer, self).__init__()
    
        self.sublayer = nn.Sequential(
                            Residual_Connection(layer), 
                            nn.LayerNorm()
        )
        
    def forward(self, x):
        x = self.sublayer(x)
        return x

In [ ]:
# unit test on sublayer

## Encoder

<img src='assets/1/encoder.PNG' /> 

In [ ]:
class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        
        self.attention = MultiHeadAttention()
        self.feed_forward = 
        
        # two sub-layers
        # 1. Self-Attention
        self.sublayer1 = Sublayer(self.attention)
          
        # 2. Feedforward Neural Network
        self.sublayer2 = Sublayer(self.attention)
        
    def forward(self, x):
        pass
        
        
        
        
        # nn.Sequential(*[])

## Decoder

<img src='assets/1/decoder.PNG' /> 

In [ ]:
class Decoder(nn.Module):
    def __init__(self):
        super(Decoder, self).__init__()
        
        # three sub-layers
        # 1. Feedforward Neural Network
        # 2. Self-Attention
        # 3. Self-Attention
        
        # employ a residual connection around each of
        # the two sub-layers, followed by layer normalization
        
        # That is, the output of each sub-layer is
        # LayerNorm(x + Sublayer(x)), where Sublayer(x) is the function 
        # implemented by the sub-layer itself
        
    def forward(self, x):
        pass

## Transformer Model

<img src='assets/1/transformer-model.PNG' width=80% height=80%/> 

In [ ]:
# The encoders are all identical in structure (yet they do not share weights)

class Transformer(nn.Module):
    
    # N is the number of stacked layers of encoders & decoders
    def __init__(self, N=6, h=8, d_k=64, d_v=64):
        super(Transformer, self).__init__()
        
        # initialize the encoder and decoder stacks with sequential container
        encoders, decoders = [], []
        
        # stack encoders
        for i in range(N):
            encoder = Encoder()
            encoders.append(encoder)
        
        # stack decoders
        for i in range(N):
            decoder = Decoder()
            decoders.append(decoder)
            
        encoders = nn.Sequential(*encoders)
        decoders = nn.Sequential(*decoders)
        
    def forward(self):
        # pass through encoders
        
        # pass through decoders but considering same input
        pass
    
        
        
        

##  Hyperparameters

In [ ]:
# Define the model parameters

# from section 3.1 Encoder and Decoder Stacks
N = 6  # Number of layers in the encoder stack and decoder stack
d_model = 512  # Dimensionality of model layers' outputs

# from section 3.2.2 Multi-Head Attention
# d_k = d_v = d_model / h = 64
h = 8  # Number of self-attention heads
d_k = 64  # Dimensionality of the linearly projected queries and keys
d_v = 64  # Dimensionality of the linearly projected values

# from section 3.3 Position-wise Feed-Forward Network
d_ff = 2048  # Dimensionality of the inner fully connected layer


In [ ]:
# Define the training parameters

# from section 5.3 Optimizer 
beta_1 = 0.9
beta_2 = 0.98
epsilon = 1e-9
warmup_steps = 4000
# train steps is stated as 100K in the paper, 
# we will use warmup steps to train 

# from section 5.4 Regularization
dropout_rate = 0.1

## Transformers Success

### Final Notes

The implementation here is a very basic representation of how Transformers work. In practice, the Transformer model is used with batches with sequence lengths. This introduces new dimensions in the tensors discussed above, and also a lot of new calculations, complex matrix multiplications. There are a lot of improvements that can be made in the implementation here, and a lot of calculations can be further optimized.   

### Resources

1. [Illustrated Transformer](https://jalammar.github.io/illustrated-transformer/) - Blog Post
2. [Attention is All You Need](https://arxiv.org/pdf/1706.03762.pdf) - Paper